# Computing the optimal road trip across the U.S.

This notebook provides the methodology and code used in the blog post, [Computing the optimal road trip across the U.S.](http://www.randalolson.com/2015/03/08/computing-the-optimal-road-trip-across-the-u-s/)

### Notebook by [Randal S. Olson](http://www.randalolson.com)

Please see the [repository README file](https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects#license) for the licenses and usage terms for the instructional material and code in this notebook. In general, I have licensed this material so that it is as widely useable and shareable as possible.

The code in this notebook is also available as a single Python script [here](https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/OptimalRoadTripHtmlSaveAndDisplay.py) courtesy of [Andrew Liesinger](https://github.com/AndrewLiesinger).

### Required Python libraries

If you don't have Python on your computer, you can use the [Anaconda Python distribution](http://continuum.io/downloads) to install most of the Python packages you need. Anaconda provides a simple double-click installer for your convenience.

This code uses base Python libraries except for `googlemaps` and `pandas` packages. You can install these packages using `pip` by typing the following commands into your command line:

> pip install pandas

> pip install googlemaps

If you're on a Mac, Linux, or Unix machine, you may need to type `sudo` before the command to install the package with administrator privileges.

### Construct a list of road trip waypoints

The first step is to decide where you want to stop on your road trip.

Make sure you look all of the locations up on [Google Maps](http://maps.google.com) first so you have the correct address, city, state, etc. If the text you use to look up the location doesn't work on Google Maps, then it won't work here either.

Add all of your waypoints to the list below. Make sure they're formatted the same way as in the example below.

*Technical note: Due to daily usage limitations of the Google Maps API, you can only have a maximum of 70 waypoints. You will have to pay Google for an increased API limit if you want to add more waypoints.*

Next you'll have to register this script with the Google Maps API so they know who's hitting their servers with hundreds of Google Maps routing requests.

1) Enable the Google Maps Distance Matrix API on your Google account. Google explains how to do that [here](https://github.com/googlemaps/google-maps-services-python#api-keys).

2) Copy and paste the API key they had you create into the code below.

In [23]:
import googlemaps, csv
import pandas as pd

creds = pd.read_json("/Users/bryan/dev_other/google_api_creds.json")
gmaps = googlemaps.Client(key=creds['default-maps']['secret'])

In [24]:
routes_df = pd.read_csv("RoutesData.csv")

In [25]:
routes_df.head()
routes_df

,number,name,fiets_index_score,length,avg_grade,ele_gain,ele_desc,start_ele,end_ele,max_grade,rd_cond,traffic,lat,long,url,route_number,secondary_route_number
0,1,Mauna Kea Hawaii,28.95,42.5,5.9,13789,-35,3,13757,24.8,3,3,19.72672,-155.086866,"""https://www.pjammcycling.com/1.--mauna-kea--...",1504789,NaN
1,2,Haleakala Hawaii,18.52,34.8,5.6,10249,-284,14,9978,15.5,5,3,20.91605,-156.381190,"""https://www.pjammcycling.com/2.--haleakala--...",638944,NaN
2,3,Mt Washington Auto Rd New Hampshire,18.01,7.4,11.9,4655,0,1584,6268,22.0,3,5,44.28927,-71.229670,"""https://www.pjammcycling.com/3.--mt-washingt...",3237,NaN
3,4,Mauna Loa Hawaii,17.83,45.2,4.4,11008,-14,3,11004,12.9,4,3,19.72647,-155.087110,"""https://www.pjammcycling.com/4.--mauna-loa--...",6998193,NaN
4,5,Pikes Peak Colorado,17.67,24.2,6.5,7914,-171,6345,14115,13.8,5,3,38.85928,-104.919660,"""https://www.pjammcycling.com/5.--pikes-peak-...",625021,NaN
5,6,Onion Valley Rd California,14.15,12.7,8.2,5270,-83,3976,9178,13.7,3,4,36.80156,-118.201950,"""https://www.pjammcycling.com/6.--onion-valle...",4624623,NaN
6,7,Horseshoe Meadows Rd California,13.74,19.2,6.5,6490,-512,3820,10052,15.3,3,4,36.54224,-118.051510,"""https://www.pjammcycling.com/7.-horseshoe-me...",1334431,NaN
7,8,Kaloko Dr Hawaii,12.82,11.7,8.4,4985,-1,93,5082,20.7,4,4,19.69051,-156.022600,"""https://www.pjammcycling.com/8.--kaloko-dr.-...",885309,NaN
8,9,White Mountain California,12.80,20.1,6.3,6375,-214,3944,10109,19.9,4,3,37.18501,-118.252970,"""https://www.pjammcycling.com/9.--white-mount...",626925,NaN
9,10,Waipoli Rd Hawaii,12.74,19.4,8.2,6299,-23,95,6371,18.8,4,3.5,20.87158,-156.438410,"""https://www.pjammcycling.com/10.--waipoli-rd...",2917983,NaN


In [26]:
routes_df.columns = [str(x).strip(" ") for x in routes_df.columns]
routes_df = routes_df.fillna(0)
routes_df[['secondary_route_number']] = routes_df[['secondary_route_number']].astype(int)
routes_df.head()


,number,name,fiets_index_score,length,avg_grade,ele_gain,ele_desc,start_ele,end_ele,max_grade,rd_cond,traffic,lat,long,url,route_number,secondary_route_number
0,1,Mauna Kea Hawaii,28.95,42.5,5.9,13789,-35,3,13757,24.8,3,3,19.72672,-155.086866,"""https://www.pjammcycling.com/1.--mauna-kea--...",1504789,0
1,2,Haleakala Hawaii,18.52,34.8,5.6,10249,-284,14,9978,15.5,5,3,20.91605,-156.381190,"""https://www.pjammcycling.com/2.--haleakala--...",638944,0
2,3,Mt Washington Auto Rd New Hampshire,18.01,7.4,11.9,4655,0,1584,6268,22.0,3,5,44.28927,-71.229670,"""https://www.pjammcycling.com/3.--mt-washingt...",3237,0
3,4,Mauna Loa Hawaii,17.83,45.2,4.4,11008,-14,3,11004,12.9,4,3,19.72647,-155.087110,"""https://www.pjammcycling.com/4.--mauna-loa--...",6998193,0
4,5,Pikes Peak Colorado,17.67,24.2,6.5,7914,-171,6345,14115,13.8,5,3,38.85928,-104.919660,"""https://www.pjammcycling.com/5.--pikes-peak-...",625021,0


Now we're going to query the Google Maps API for the shortest route between all of the waypoints.

This is equivalent to doing Google Maps directions lookups on the Google Maps site, except now we're performing hundreds of lookups automatically using code.

If you get an error on this part, that most likely means one of the waypoints you entered couldn't be found on Google Maps. Another possible reason for an error here is if it's not possible to drive between the points, e.g., finding the driving directions between Hawaii and Florida will return an error until we invent flying cars.

### Gather the distance traveled on the shortest route between all waypoints

In [27]:
bike_waypoints = [(row['name'].strip(" "), (row['lat'], row['long'])) for index, row in routes_df.iterrows() if 'Hawaii' not in row['name']]

In [28]:
lat_long_lookup = {}
for x in bike_waypoints:
    lat_long_lookup[x[0]] = x[1]
print lat_long_lookup

{'Trail Ridge Rd Colorado': (40.39628, -105.56513000000001), 'Hwy 153 Utah': (38.2784, -112.60248), 'Hwy 21-Whittaker Forest California': (36.56638, -119.02566000000002), 'Shirley Meadows California': (35.7068, -118.4567), 'Glacier Lodge Rd California': (37.16608, -118.30093000000001), 'Left Hand Canyon Colorado': (40.13348, -105.28528999999999), 'Mt Hood Oregon': (45.329879999999996, -121.91116000000001), 'Mt Lemmon Arizona': (32.29483, -110.75438), 'Tollhouse Rd California': (37.01812, -119.39948000000001), 'Burke Mt Vermont': (44.59102, -71.91734), 'Pine Creek Rd California': (37.41956, -118.58353999999999), 'Grand Mesa North Colorado': (39.1907, -108.14116299999999), 'Hwy 14 West Wyoming': (44.57073, -107.70719), 'Wolf Creek Pass Colorado': (37.435590000000005, -106.88611999999999), 'Whitney Portal California': (36.601859999999995, -118.0738), 'Hwy 82 New Mexico': (32.95855, -105.94006), 'East Portal Colorado': (38.541154, -107.69323), 'Kolob Terrace Rd Utah': (37.25145, -113.14088

In [29]:
route_num_lookup = {}
for index, row in routes_df.iterrows():
    route_num_lookup[row['name'].strip(" ")] = row['route_number']
print route_num_lookup

{'Trail Ridge Rd Colorado': 9702391, 'Hwy 153 Utah': 9834456, 'Hwy 21-Whittaker Forest California': 7193411, 'Shirley Meadows California': 9985856, 'Glacier Lodge Rd California': 9664095, 'Left Hand Canyon Colorado': 10022984, 'Mt Hood Oregon': 4894953, 'Mt Lemmon Arizona': 8950048, 'Tollhouse Rd California': 631922, 'Burke Mt Vermont': 130445, 'Pine Creek Rd California': 4350313, 'Grand Mesa North Colorado': 1798960, 'Hwy 14 West Wyoming': 1653954, 'Wolf Creek Pass Colorado': 3650413, 'Whitney Portal California': 2318996, 'Hwy 82 New Mexico': 678275, 'Kaloko Dr Hawaii': 885309, 'Kolob Terrace Rd Utah': 9776483, 'Grand Mesa South Colorado': 1211716, 'Portuguese Pass California': 94534, 'Hwy 21-245-180-198 California': 14594861, 'Cottonwood Pass Colorado': 1515689, 'Mt Equinox Vermont': 3687, 'Nine Mile Rd California': 623599, 'Lake Sabrina California': 4664092, 'Monitor Pass East California': 4235370, 'Balch Park Rd California': 10008833, 'Baldwin/Olinda Rd Hawaii': 8964831, 'Parks Cre

In [30]:
gmaps.distance_matrix(origins=[(35.96737, -118.48555)], destinations=[(34.13577606182823,-117.19128549098969)],
                                      mode="bicycling", # Change this to "walking" for walking directions,
                                                      # "bicycling" for biking directions, etc.
                                      language="English",
                                      units="metric")
# origins = (journeydf['Start Latitude'][i], journeydf['Start Longitude'][i])
# destinations = (journeydf['End Latitude'][i], journeydf['End Longitude'][i])

{u'destination_addresses': [u'4304 Highland Ave, Highland, CA 92346, USA'],
 u'origin_addresses': [u'Forest Rte 22S05, California, USA'],
 u'rows': [{u'elements': [{u'distance': {u'text': u'316 km', u'value': 315768},
     u'duration': {u'text': u'17 hours 40 mins', u'value': 63622},
     u'status': u'OK'}]}],
 u'status': u'OK'}

In [10]:
make_api_calls = False
if make_api_calls:
    from itertools import combinations

    waypoint_distances = {}
    waypoint_durations = {}
    current_waypoint=""

    for (waypoint1, waypoint2) in combinations(bike_waypoints, 2):
    #     print waypoint1, waypoint2
        try:
            if current_waypoint != waypoint1:
                print waypoint1
                current_waypoint = waypoint1
            route = gmaps.distance_matrix(origins=[waypoint1[1]],
                                          destinations=[waypoint2[1]],
                                          mode="bicycling", # Change this to "walking" for walking directions,
                                                          # "bicycling" for biking directions, etc.
                                          language="English",
                                          units="metric")
        #     print route

        #         # "distance" is in meters
            distance = route["rows"][0]["elements"][0]["distance"]["value"]

        #         # "duration" is in seconds
            duration = route["rows"][0]["elements"][0]["duration"]["value"]

            waypoint_distances[frozenset([waypoint1[0], waypoint2[0]])] = distance
            waypoint_durations[frozenset([waypoint1[0], waypoint2[0]])] = duration

        except Exception as e:
            print("Error with finding the route between %s and %s." % (waypoint1, waypoint2))
            print e

Now that we have the routes between all of our waypoints, let's save them to a text file so we don't have to bother Google about them again.

In [12]:
if make_api_calls:
    with open("my-gps-waypoints-dist-dur.tsv", "w") as out_file:
        out_file.write("\t".join(["waypoint1",
                                  "waypoint2",
                                  "distance_m",
                                  "duration_s"]))

        for (waypoint1, waypoint2) in waypoint_distances.keys():
            out_file.write("\n" +
                           "\t".join([waypoint1,
                                      waypoint2,
                                      str(waypoint_distances[frozenset([waypoint1, waypoint2])]),
                                      str(waypoint_durations[frozenset([waypoint1, waypoint2])])]))

### Use a genetic algorithm to optimize the order to visit the waypoints in

Instead of exhaustively looking at every possible solution, genetic algorithms start with a handful of random solutions and continually tinkers with these solutions — always trying something slightly different from the current solutions and keeping the best ones — until they can’t find a better solution any more.

Below, all you need to do is make sure that the file name above matches the file name below (both currently `my-waypoints-dist-dur.tsv`) and run the code. The code will read in your route information and use a genetic algorithm to discover an optimized driving route.

In [17]:
import pandas as pd
import numpy as np

rebuild_genetic_seeds=False

if rebuild_genetic_seeds:
    waypoint_distances = {}
    waypoint_durations = {}
    all_waypoints = set()

    waypoint_data = pd.read_csv("my-gps-waypoints-dist-dur.tsv", sep="\t")

    for i, row in waypoint_data.iterrows():
        waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
        waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
        all_waypoints.update([row.waypoint1, row.waypoint2])

In [18]:
import random

def compute_fitness(solution):
    """
        This function returns the total distance traveled on the current road trip.
        
        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """
    
    solution_fitness = 0.0
    
    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
    return solution_fitness

def generate_random_agent():
    """
        Creates a random road trip from the waypoints.
    """
    
    new_random_agent = list(all_waypoints)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.
        
        A point mutation swaps the order of two waypoints in the road trip.
    """
    
    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)
    
    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
            
    return tuple(agent_genome)

def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.
        
        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """
    
    agent_genome = list(agent_genome)
    
    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)
    
    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]
    
    return tuple(agent_genome)

def generate_random_population(pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """
    
    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population
    
def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.
        
        `generations` and `population_size` must be a multiple of 10.
    """
    
    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    
    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(population_size)

    # For `generations` number of repetitions...
    for generation in range(generations):
        
        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):
            
            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                print(agent_genome)
                print("")

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(4):
                new_population.append(mutate_agent(agent_genome, 3))
                
            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring 
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population

Try running the genetic algorithm a few times to see the different solutions it comes up with. It should take about a minute to finish running.

In [19]:
if rebuild_genetic_seeds:
    run_genetic_algorithm(generations=10000, population_size=100)

In [31]:
ouput_list = ['Beartooth Pass North Montana', 'Hwy 14 Alt Route Wyoming', 'Hwy 14 West Wyoming', 'Guanella Pass Colorado', 'Mt Evans Colorado', 'Magnolia Dr Colorado', 'Left Hand Canyon Colorado', 'Trail Ridge Rd Colorado', 'Pikes Peak Colorado', 'Cottonwood Pass Colorado', 'Independence Pass Colorado', 'Slumgullion Pass West Colorado', 'Wolf Creek Pass Colorado', 'East Portal Colorado', 'Grand Mesa South Colorado', 'Grand Mesa North Colorado', 'Hwy 31 West Utah', 'Wheeler Peak Nevada', 'Hwy 156-158 Nevada', 'Hwy 157 Mt Charleston Nevada', 'Hwy 157-158 Nevada', 'Hwy 156-Lee Canyon Nevada', 'Dantes View California', 'Daylight Pass Rd California', 'Townes Pass East California', 'Emigrant Pass East California', 'Townes Pass West California', 'Wildrose California', 'Hwy 18 North California', 'N4/Table Mt California', 'Lone Pine-Table Mountain California', 'Hwy 18-Waterman Canyon California', 'Hwy 330 California', 'Hwy 38-Valley of Falls California', 'Palomar Mt California', 'Nate Harrison Grade California', 'Mt Baldy California', 'Hwy 39-Crystal Lake Rd California', "Hwy 39-Dawson's Saddle California", 'Refugio Rd California', 'Gibraltar Road Santa Barbara California', 'Hwy 190 California', 'Bear Creek Road California', 'Balch Park Rd California', 'Mineral King Rd California', 'Hwy 198 California', 'Hwy 21-Whittaker Forest California', 'Hwy 21-245-180-198 California', 'Hwy 180 West California', 'Tollhouse Rd California', 'Beasore Rd California', 'Hwy 168 California', 'Portuguese Pass California', 'Sherman Pass West California', 'Shirley Meadows California', 'Breckenridge Rd East California', 'Nine Mile Rd California', 'Horseshoe Meadows Rd California', 'Whitney Portal California', 'Onion Valley Rd California', 'Glacier Lodge Rd California', 'White Mountain California', 'Death Valley Rd East California', 'South Lake California', 'Lake Sabrina California', 'Pine Creek Rd California', 'Rock Creek California', 'Sonora Pass West California', 'Monitor Pass East California', 'Mt Rose California', 'Mosquito Ridge Rd East California', 'Mix Canyon Fairfield California', 'Etna Summit West California', 'Mt Shasta California', 'Parks Creek East California', 'Mt Ashland Oregon', 'Mt Hood Oregon', 'Hurricane Ridge Washington', 'Mission Ridge Washington', 'Mt Spokane Washington', 'Elkhorn Summit Oregon', 'Mt Harrision Idaho', 'Powder Mountain Utah', 'Big Cottonwood Canyon Utah', 'Little Cottonwood Canyon Utah', 'Guardsman Pass Utah', 'Empire Pass Utah', 'Hwy 153 Utah', 'Hwy 143 Utah', 'Kolob Terrace Rd Utah', 'Mt Lemmon Arizona', 'Mt Graham Arizona', 'Sandia Crest Hwy New Mexico', 'Hwy 82 New Mexico', "Clingman's Dome Tennessee", 'Mt Auscutney Vermont', 'Mt Equinox Vermont', 'Mt Washington Auto Rd New Hampshire', 'Burke Mt Vermont', 'Whiteface Mt New York'][::-1]

In [32]:
print ouput_list

['Whiteface Mt New York', 'Burke Mt Vermont', 'Mt Washington Auto Rd New Hampshire', 'Mt Equinox Vermont', 'Mt Auscutney Vermont', "Clingman's Dome Tennessee", 'Hwy 82 New Mexico', 'Sandia Crest Hwy New Mexico', 'Mt Graham Arizona', 'Mt Lemmon Arizona', 'Kolob Terrace Rd Utah', 'Hwy 143 Utah', 'Hwy 153 Utah', 'Empire Pass Utah', 'Guardsman Pass Utah', 'Little Cottonwood Canyon Utah', 'Big Cottonwood Canyon Utah', 'Powder Mountain Utah', 'Mt Harrision Idaho', 'Elkhorn Summit Oregon', 'Mt Spokane Washington', 'Mission Ridge Washington', 'Hurricane Ridge Washington', 'Mt Hood Oregon', 'Mt Ashland Oregon', 'Parks Creek East California', 'Mt Shasta California', 'Etna Summit West California', 'Mix Canyon Fairfield California', 'Mosquito Ridge Rd East California', 'Mt Rose California', 'Monitor Pass East California', 'Sonora Pass West California', 'Rock Creek California', 'Pine Creek Rd California', 'Lake Sabrina California', 'South Lake California', 'Death Valley Rd East California', 'White 

In [33]:
[lat_long_lookup[x] for x in ouput_list]

[(44.38987, -73.8223),
 (44.59102, -71.91734),
 (44.28927, -71.22967),
 (43.11743, -73.11345),
 (43.43782, -72.40574000000001),
 (35.68472, -83.53517),
 (32.95855, -105.94006),
 (35.163109999999996, -106.34845),
 (32.718379999999996, -109.72583),
 (32.29483, -110.75438),
 (37.25145, -113.14088999999998),
 (37.83863, -112.81978000000001),
 (38.2784, -112.60248),
 (40.529109999999996, -111.47927),
 (40.53862, -111.48375),
 (40.57271, -111.77646999999999),
 (40.61933, -111.78945),
 (41.32072, -111.82961),
 (42.38671, -113.55081000000001),
 (44.99366, -118.08200000000001),
 (47.83, -117.20200000000001),
 (47.39676, -120.30286000000001),
 (48.112970000000004, -123.41876),
 (45.329879999999996, -121.91116000000001),
 (42.13153, -122.61225),
 (41.43565, -122.46185),
 (41.32735, -122.3223),
 (41.34585, -123.04316000000001),
 (38.41062, -122.05215),
 (39.02411, -120.72053000000001),
 (39.4027, -119.74638),
 (38.64263, -119.52776000000001),
 (38.32459, -119.75218000000001),
 (37.46311, -118.5932

In [34]:
[(x, "https://www.strava.com/segments/" + str(route_num_lookup[x])) for x in ouput_list]

[('Whiteface Mt New York', 'https://www.strava.com/segments/667415'),
 ('Burke Mt Vermont', 'https://www.strava.com/segments/130445'),
 ('Mt Washington Auto Rd New Hampshire',
  'https://www.strava.com/segments/3237'),
 ('Mt Equinox Vermont', 'https://www.strava.com/segments/3687'),
 ('Mt Auscutney Vermont', 'https://www.strava.com/segments/229501'),
 ("Clingman's Dome Tennessee", 'https://www.strava.com/segments/629572'),
 ('Hwy 82 New Mexico', 'https://www.strava.com/segments/678275'),
 ('Sandia Crest Hwy New Mexico', 'https://www.strava.com/segments/1466117'),
 ('Mt Graham Arizona', 'https://www.strava.com/segments/623710'),
 ('Mt Lemmon Arizona', 'https://www.strava.com/segments/8950048'),
 ('Kolob Terrace Rd Utah', 'https://www.strava.com/segments/9776483'),
 ('Hwy 143 Utah', 'https://www.strava.com/segments/9657320'),
 ('Hwy 153 Utah', 'https://www.strava.com/segments/9834456'),
 ('Empire Pass Utah', 'https://www.strava.com/segments/4433534'),
 ('Guardsman Pass Utah', 'https://ww

### Visualize your road trip on a Google map

Now that we have an ordered list of the waypoints, we should put them on a Google map so we can see the trip from a high level and make any extra adjustments.

There's no easy way to make this visualization in Python, but the Google Maps team provides a nice JavaScript library for visualizing routes on a Google Map.

Here's an example map with the route between 50 waypoints visualized: [link](http://rhiever.github.io/optimal-roadtrip-usa/major-landmarks.html)

The tricky part here is that the JavaScript library only plots routes with a maximum of 10 waypoints. If we want to plot a route with >10 waypoints, we need to call the route plotting function multiple times.

Thanks to some optimizations by [Nicholas Clarke](https://github.com/nicholasgodfreyclarke) to my original map, this is a simple operation:

1) Copy the final route generated by the genetic algorithm above.

2) Place brackets (`[` & `]`) around the route, e.g.,

    ['Graceland, Elvis Presley Boulevard, Memphis, TN', 'Vicksburg National Military Park, Clay Street, Vicksburg, MS', 'French Quarter, New Orleans, LA', 'USS Alabama, Battleship Parkway, Mobile, AL', 'Cape Canaveral, FL', 'Okefenokee Swamp Park, Okefenokee Swamp Park Road, Waycross, GA', "Fort Sumter National Monument, Sullivan's Island, SC", 'Wright Brothers National Memorial Visitor Center, Manteo, NC', 'Congress Hall, Congress Place, Cape May, NJ 08204', 'Shelburne Farms, Harbor Road, Shelburne, VT', 'Omni Mount Washington Resort, Mount Washington Hotel Road, Bretton Woods, NH', 'Acadia National Park, Maine', 'USS Constitution, Boston, MA', 'The Breakers, Ochre Point Avenue, Newport, RI', 'The Mark Twain House & Museum, Farmington Avenue, Hartford, CT', 'Statue of Liberty, Liberty Island, NYC, NY', 'Liberty Bell, 6th Street, Philadelphia, PA', 'New Castle Historic District, Delaware', 'Maryland State House, 100 State Cir, Annapolis, MD 21401', 'White House, Pennsylvania Avenue Northwest, Washington, DC', 'Mount Vernon, Fairfax County, Virginia', 'Lost World Caverns, Lewisburg, WV', 'Olympia Entertainment, Woodward Avenue, Detroit, MI', 'Spring Grove Cemetery, Spring Grove Avenue, Cincinnati, OH', 'Mammoth Cave National Park, Mammoth Cave Pkwy, Mammoth Cave, KY', 'West Baden Springs Hotel, West Baden Avenue, West Baden Springs, IN', 'Gateway Arch, Washington Avenue, St Louis, MO', 'Lincoln Home National Historic Site Visitor Center, 426 South 7th Street, Springfield, IL', 'Taliesin, County Road C, Spring Green, Wisconsin', 'Fort Snelling, Tower Avenue, Saint Paul, MN', 'Terrace Hill, Grand Avenue, Des Moines, IA', 'C. W. Parker Carousel Museum, South Esplanade Street, Leavenworth, KS', 'Ashfall Fossil Bed, Royal, NE', 'Mount Rushmore National Memorial, South Dakota 244, Keystone, SD', 'Fort Union Trading Post National Historic Site, Williston, North Dakota 1804, ND', 'Glacier National Park, West Glacier, MT', 'Yellowstone National Park, WY 82190', 'Craters of the Moon National Monument & Preserve, Arco, ID', 'Hanford Site, Benton County, WA', 'Columbia River Gorge National Scenic Area, Oregon', 'Cable Car Museum, 94108, 1201 Mason St, San Francisco, CA 94108', 'San Andreas Fault, San Benito County, CA', 'Hoover Dam, NV', 'Grand Canyon National Park, Arizona', 'Bryce Canyon National Park, Hwy 63, Bryce, UT', 'Pikes Peak, Colorado', 'Carlsbad Caverns National Park, Carlsbad, NM', 'The Alamo, Alamo Plaza, San Antonio, TX', 'Chickasaw National Recreation Area, 1008 W 2nd St, Sulphur, OK 73086', 'Toltec Mounds, Scott, AR']

3) Paste the final route with brackets into [line 93](https://github.com/rhiever/optimal-roadtrip-usa/blob/gh-pages/major-landmarks.html#L93) of my road trip map code. It should look like this:

    optimal_route = [ ... ]
    
where `...` is your optimized road trip.

That's all it takes! Now you have your own optimized road trip ready to show off to the world.

### Some technical notes

As I mentioned in the [original article](http://www.randalolson.com/2015/03/08/computing-the-optimal-road-trip-across-the-u-s/), by the end of 5,000 generations, the genetic algorithm will very likely find a *good* but probably not the *absolute best* solution to the optimal routing problem. It is in the nature of genetic algorithms that we never know if we found the absolute best solution.

However, there exist some brilliant analytical solutions to the optimal routing problem such as the [Concorde TSP solver](http://en.wikipedia.org/wiki/Concorde_TSP_Solver). If you're interested in learning more about Concorde and how it's possible to find a perfect solution to the routing problem, I advise you check out [Bill Cook's article](http://www.math.uwaterloo.ca/tsp/usa50/index.html) on the topic.

### If you have any questions

Please feel free to:

* [Email me](http://www.randalolson.com/contact/),

* [Tweet](https://twitter.com/randal_olson) at me, or

* comment on the [blog post](http://www.randalolson.com/2015/03/08/computing-the-optimal-road-trip-across-the-u-s/)

I'm usually pretty good about getting back to people within a day or two.